In [1]:
import os
import sys
# os.chdir("../")
sys.path.append('../tsp_nls')
sys.path.append('../tsp')
sys.path.append("..")  # This points to `DeepACO`

TSPLIB_DIR = "../data/tsp/TSPLIB"
tsp_files = [f for f in os.listdir(TSPLIB_DIR) if f.endswith(".tsp")]
print(tsp_files)

print("Current Working Directory:", os.getcwd())
import torch
from torch.distributions import Categorical, kl
# from d2l.torch import Animator

from tsp_nls.greedy import test_greedy, GreedyTSP

torch.manual_seed(12345)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

['pr439.tsp', 'rd100.tsp', 'rl5934.tsp', 'pcb442.tsp', 'u2319.tsp', 'gil262.tsp', 'pcb3038.tsp', 'lin105.tsp', 'fl417.tsp', 'tsp225.tsp', 'fl1400.tsp', 'nrw1379.tsp', 'd2103.tsp', 'kroA150.tsp', 'pcb1173.tsp', 'd198.tsp', 'fl1577.tsp', 'ch130.tsp', 'kroB100.tsp', 'u1060.tsp', 'berlin52.tsp', 'eil51.tsp', 'rl1304.tsp', 'u2152.tsp', 'u724.tsp', 'kroD100.tsp', 'pr299.tsp', 'rd400.tsp', 'vm1084.tsp', 'rat575.tsp', 'd1655.tsp', 'ch150.tsp', 'd15112.tsp', 'pr107.tsp', 'kroB200.tsp', 'brd14051.tsp', 'a280.tsp', 'd1291.tsp', 'pr264.tsp', 'pr76.tsp', 'd493.tsp', 'pr136.tsp', 'rat195.tsp', 'rl11849.tsp', 'kroA100.tsp', 'kroB150.tsp', 'bier127.tsp', 'kroC100.tsp', 'usa13509.tsp', 'eil76.tsp', 'pr124.tsp', 'rl1323.tsp', 'p654.tsp', 'rl1889.tsp', 'd657.tsp', 'eil101.tsp', 'fnl4461.tsp', 'pr2392.tsp', 'rat783.tsp', 'ts225.tsp', 'u1432.tsp', 'u1817.tsp', 'lin318.tsp', 'd18512.tsp', 'rl5915.tsp', 'st70.tsp', 'rat99.tsp', 'fl3795.tsp', 'u159.tsp', 'kroA200.tsp', 'u574.tsp', 'pr1002.tsp', 'pr152.tsp', '

# Test Performance of DeepACO

## TSP500

### Authors provided

In [18]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 500
n_ants = 50
t_aco = [2, 12]
k_sparse = 50
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=500

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# # Genetic Algorithm
# print('-------Genetic Algorithm-------')
# import tsp.genetic as genetic

# pop_size = 500
# crossover_rate = 0.85
# mutation_rate=0.15
# elitism_count=2
# tournament_size=5
# # t_ga=[500, 1000] # best performance
# t_ga=[100] # limited runtime

# ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
#     test_list, 
#     pop_size=pop_size, 
#     crossover_rate=crossover_rate, 
#     mutation_rate=mutation_rate, 
#     elitism_count=elitism_count, 
#     tournament_size=tournament_size, 
#     t_ga=t_ga, 
#     device=device
# )

# print('Total duration: ', duration_ga)
# for i, t in enumerate(t_ga):
#     print(f"T={t}, average cost is {avg_ga_best[i]}.")
#     res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

-------ACO Model (LS)-------


100%|██████████| 128/128 [15:48<00:00,  7.41s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/3555379313.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Total duration:  948.8780181407928
T=2, average cost is 17.70496368408203.
T=12, average cost is 17.522695541381836.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp500.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [04:16<00:00,  2.00s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/3555379313.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

total duration:  256.3679599761963
T=2, average cost is 17.07706069946289.
T=12, average cost is 16.96156120300293.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp500.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [20:08<00:00,  9.44s/it]


total duration:  1208.8285551071167
T=2, average cost is 16.931360244750977.
T=12, average cost is 16.838075637817383.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  0.5012819766998291
Average cost is 20.80839306605617.


## Our reproduction

In [2]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 500
n_ants = 50
t_aco = [2, 12]
k_sparse = 50
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=500

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# # Genetic Algorithm
# print('-------Genetic Algorithm-------')
# import tsp.genetic as genetic

# pop_size = 500
# crossover_rate = 0.85
# mutation_rate=0.15
# elitism_count=2
# tournament_size=5
# # t_ga=[500, 1000] # best performance
# t_ga=[100] # limited runtime

# ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
#     test_list, 
#     pop_size=pop_size, 
#     crossover_rate=crossover_rate, 
#     mutation_rate=mutation_rate, 
#     elitism_count=elitism_count, 
#     tournament_size=tournament_size, 
#     t_ga=t_ga, 
#     device=device
# )

# print('Total duration: ', duration_ga)
# for i, t in enumerate(t_ga):
#     print(f"T={t}, average cost is {avg_ga_best[i]}.")
#     res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
# print('-------ACO Model (LS)-------')
# avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
# print('Total duration: ', duration)
# for i, t in enumerate(t_aco):
#     res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
#     print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'cost_tsp{test_n_node}_reproduced.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

/Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/project/../tsp_nls/utils.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_tensor = torch.load(filename)


-------ACO Model (LS)-------


100%|██████████| 128/128 [14:08<00:00,  6.63s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_72612/496587869.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

Total duration:  848.2977397441864
T=2, average cost is 17.7020206451416.
T=12, average cost is 17.523035049438477.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp500-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [03:54<00:00,  1.83s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_72612/496587869.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

total duration:  234.34918999671936
T=2, average cost is 17.16456413269043.
T=12, average cost is 17.00470733642578.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp500-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [17:59<00:00,  8.43s/it]


total duration:  1079.3477549552917
T=2, average cost is 17.051776885986328.
T=12, average cost is 16.91356658935547.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  0.5416841506958008
Average cost is 20.80839306605617.


## TSP1000

In [ ]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 1000
n_ants = 50
t_aco = [2]
k_sparse = 50
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=1000

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# # Genetic Algorithm
# print('-------Genetic Algorithm-------')
# import tsp.genetic as genetic

# pop_size = 500
# crossover_rate = 0.85
# mutation_rate=0.15
# elitism_count=2
# tournament_size=5
# # t_ga=[500, 1000] # best performance
# t_ga=[100] # limited runtime

# ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
#     test_list, 
#     pop_size=pop_size, 
#     crossover_rate=crossover_rate, 
#     mutation_rate=mutation_rate, 
#     elitism_count=elitism_count, 
#     tournament_size=tournament_size, 
#     t_ga=t_ga, 
#     device=device
# )

# print('Total duration: ', duration_ga)
# for i, t in enumerate(t_ga):
#     print(f"T={t}, average cost is {avg_ga_best[i]}.")
#     res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
# print('-------ACO Model (LS)-------')
# avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
# print('Total duration: ', duration)
# for i, t in enumerate(t_aco):
#     res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
#     print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))
    
# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

-------ACO Model (LS)-------


100%|██████████| 128/128 [31:57<00:00, 14.98s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/1465468906.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Total duration:  1917.347046136856
T=2, average cost is 25.049863815307617.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp1000.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [06:35<00:00,  3.09s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/1465468906.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

total duration:  395.95121693611145
T=2, average cost is 24.173004150390625.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp1000.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [16:33<00:00,  7.77s/it]


total duration:  993.9642057418823
T=2, average cost is 24.05609703063965.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  1.131572961807251
Average cost is 28.902860519890396.


## Our reproduction

In [3]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 1000
n_ants = 100
t_aco = [2, 12]
k_sparse = 100
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=1000

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# # Genetic Algorithm
# print('-------Genetic Algorithm-------')
# import tsp.genetic as genetic

# pop_size = 500
# crossover_rate = 0.85
# mutation_rate=0.15
# elitism_count=2
# tournament_size=5
# # t_ga=[500, 1000] # best performance
# t_ga=[100] # limited runtime

# ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
#     test_list, 
#     pop_size=pop_size, 
#     crossover_rate=crossover_rate, 
#     mutation_rate=mutation_rate, 
#     elitism_count=elitism_count, 
#     tournament_size=tournament_size, 
#     t_ga=t_ga, 
#     device=device
# )

# print('Total duration: ', duration_ga)
# for i, t in enumerate(t_ga):
#     print(f"T={t}, average cost is {avg_ga_best[i]}.")
#     res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
# print('-------ACO Model (LS)-------')
# avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
# print('Total duration: ', duration)
# for i, t in enumerate(t_aco):
#     res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
#     print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'cost_tsp{test_n_node}_reproduced.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_72612/2947436511.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_nls.load_state_dict(torch.load(model_fil

-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp1000-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [29:21<00:00, 13.76s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_72612/2947436511.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

total duration:  1761.2105638980865
T=2, average cost is 24.186599731445312.
T=12, average cost is 23.970861434936523.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp1000-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [1:28:14<00:00, 41.36s/it]


total duration:  5294.3274211883545
T=2, average cost is 24.059968948364258.
T=12, average cost is 23.863157272338867.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  1.160876989364624
Average cost is 28.902860519890396.


# Test Generalization performance of DeepACO across TSP scales

## Test TSP500

In [13]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2, 10]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=500

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

ga_res, avg_ga_best, duration_ga = genetic.test_genetic(
    test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))


# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'generalization_cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

/Users/erostrate9/Desktop/CSI5137B test/project/code/DeepACO/project/../tsp_nls/utils.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_tensor = torch.load(filename)


-------Genetic Algorithm-------


Running Genetic Algorithm...: 100%|██████████| 128/128 [07:55<00:00,  3.72s/it]


Total duration:  475.86718797683716
T=100, average cost is 150.9008331298828.
-------ACO Model (LS)-------


100%|██████████| 128/128 [06:41<00:00,  3.13s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/1757052597.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

Total duration:  401.16883397102356
T=2, average cost is 17.7701416015625.
T=10, average cost is 17.6475830078125.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [01:44<00:00,  1.22it/s]


total duration:  104.9275450706482
T=2, average cost is 17.21586036682129.
T=10, average cost is 17.043716430664062.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  0.4912858009338379
Average cost is 20.80839306605617.


In [20]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2, 10]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=500

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))


/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/209778338.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_nls.load_state_dict(torch.load(model_file

-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [07:34<00:00,  3.55s/it]

total duration:  454.75535893440247
T=2, average cost is 17.12049102783203.
T=10, average cost is 16.970735549926758.


## Test TSP1000

In [15]:
from tsp_nls.utils import load_test_dataset
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)
test_n_node=1000

model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

test_list = load_test_dataset(test_n_node, k_sparse, device, start_node = 0)
res_avg_cost = {}

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

res, avg_ga_best, duration_ga = genetic.test_genetic(
    test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
print(res)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)


import csv
RES_DIR = 'results/'
f_out = f'generalization_cost_tsp{test_n_node}.csv'
test_set = f'TSP{test_n_node}'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

-------Genetic Algorithm-------


Running Genetic Algorithm...: 100%|██████████| 128/128 [11:07<00:00,  5.21s/it]


Total duration:  667.4712688922882
[tensor([358.375]), tensor([363.750]), tensor([358.387]), tensor([361.575]), tensor([349.389]), tensor([352.195]), tensor([364.221]), tensor([361.568]), tensor([356.839]), tensor([350.290]), tensor([348.650]), tensor([359.300]), tensor([354.123]), tensor([340.603]), tensor([356.796]), tensor([350.264]), tensor([363.149]), tensor([358.517]), tensor([357.980]), tensor([358.687]), tensor([362.939]), tensor([362.833]), tensor([361.490]), tensor([357.987]), tensor([364.790]), tensor([352.824]), tensor([366.269]), tensor([355.948]), tensor([355.555]), tensor([355.396]), tensor([358.204]), tensor([351.430]), tensor([352.927]), tensor([357.475]), tensor([355.605]), tensor([354.217]), tensor([360.268]), tensor([354.864]), tensor([353.113]), tensor([365.204]), tensor([356.351]), tensor([363.800]), tensor([354.963]), tensor([361.426]), tensor([362.298]), tensor([354.078]), tensor([354.292]), tensor([359.647]), tensor([350.498]), tensor([355.461]), tensor([363.92

100%|██████████| 128/128 [13:44<00:00,  6.44s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/646809438.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

Total duration:  824.4318702220917
T=2, average cost is 25.132539749145508.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [03:59<00:00,  1.87s/it]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/646809438.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

total duration:  239.235506772995
T=2, average cost is 24.45579719543457.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 128
device: cpu


100%|██████████| 128/128 [08:57<00:00,  4.20s/it]


total duration:  537.5747299194336
T=2, average cost is 24.344730377197266.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  1.1108489036560059
Average cost is 28.902860519890396.


## Test TSPLIB

In [6]:
len(tsplib_test_list)

8

In [ ]:
from tsp_nls.utils import load_TSPLIB_test_instance
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2, 10]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)

# model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'
model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

TSPLIB_DIR = '../data/tsp/TSPLIB'
file_names=['bier127.tsp', 'ch130.tsp', 'ch150.tsp', 'eil101.tsp', 'kroA100.tsp', 'kroA150.tsp', 'kroB100.tsp', 'kroC100.tsp',
            'kroD100.tsp', 'kroE100.tsp', 'lin105.tsp', 'pr107.tsp', 'pr124.tsp', 'rd100.tsp']
tsplib_test_list = load_TSPLIB_test_instance(dir=TSPLIB_DIR, file_names=file_names, device=device, start_node=0, k_sparse = k_sparse, normalized=True)
res_avg_cost = {}

    
# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(tsplib_test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(tsplib_test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(tsplib_test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(tsplib_test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

res, avg_ga_best, duration_ga = genetic.test_genetic(
    tsplib_test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

import csv
RES_DIR = 'results/'
f_out = 'generalization_cost_tsplib100_14.csv'
test_set = 'TSPLIB100'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

-------ACO Model (LS)-------


100%|██████████| 8/8 [00:00<00:00, 14.76it/s]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_72612/62463609.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

Total duration:  0.5433483123779297
T=2, average cost is 6.89272403717041.
T=10, average cost is 6.8211469650268555.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 8
device: cpu


100%|██████████| 8/8 [00:00<00:00, 22.40it/s]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_72612/62463609.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

total duration:  0.357973575592041
T=2, average cost is 6.812536716461182.
T=10, average cost is 6.774055480957031.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 8
device: cpu


100%|██████████| 8/8 [00:02<00:00,  3.22it/s]


total duration:  2.4848527908325195
T=2, average cost is 6.76121187210083.
T=10, average cost is 6.725982666015625.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  0.008374691009521484
Average cost is 8.316447482560761.
-------Genetic Algorithm-------


Running Genetic Algorithm...: 100%|██████████| 8/8 [00:22<00:00,  2.80s/it]

Total duration:  22.399849891662598
T=100, average cost is 214.61898803710938.


In [ ]:
from tsp_nls.utils import load_TSPLIB_test_instance
from tsp_nls.test import test
import tsp_nls.net as net
# import tsp.test_utils as test_aco

# Configuration
pretrain_n_node = 100
n_ants = 20
t_aco = [2, 10]
k_sparse = 20
torch.set_printoptions(precision=3, sci_mode=False)

# model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}.pt'
model_file = f'../pretrained/tsp_nls/tsp{pretrain_n_node}-best.pt'

TSPLIB_DIR = '../data/tsp/TSPLIB'
file_names=['bier127.tsp', 'ch130.tsp', 'ch150.tsp', 'eil101.tsp']
tsplib_test_list = load_TSPLIB_test_instance(dir=TSPLIB_DIR, file_names=file_names, device=device, start_node=0, k_sparse = k_sparse, normalized=True)
res_avg_cost = {}

    
# Testing ACO (LS) Model
# avg_aco_best, duration = test_aco.test(tsplib_test_list, None, n_ants, t_aco, k_sparse)
print('-------ACO Model (LS)-------')
avg_aco_best, duration = test(tsplib_test_list, None, n_ants, t_aco, k_sparse, local_search='2opt')
print('Total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'ACO(LS)(T={t})'] = float(avg_aco_best[i])
    print(f"T={t}, average cost is {avg_aco_best[i]}.")
    
# Testing Deep ACO (LS) Model 
print('-------DeepACO (LS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(tsplib_test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_nls, n_ants, t_aco, k_sparse, local_search='2opt')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(LS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Testing Deep ACO Model (NLS)
print('-------DeepACO (NLS)-------')
print("checkpoint:", model_file)
print("number of instances:", len(tsplib_test_list))
print("device:", 'cpu' if device == 'cpu' else device+"+cpu" )

net_nls = net.Net().to(device)
net_nls.load_state_dict(torch.load(model_file, map_location=device))
avg_aco_best, duration = test(tsplib_test_list, net_nls, n_ants, t_aco, k_sparse, local_search='nls')
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    res_avg_cost[f'DeepACO(NLS)(T={t})'] = float(avg_aco_best[i])
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

# Greedy
greedy_tsp = GreedyTSP(device=device)
avg_greedy_cost, greedy_duration = test_greedy(tsplib_test_list, greedy_tsp, show_progress=False)
print('--------Greedy Algorithm:--------')
print('Total duration: ', greedy_duration)
print(f"Average cost is {avg_greedy_cost}.")
res_avg_cost[f'Greedy'] = float(avg_greedy_cost)

# Genetic Algorithm
print('-------Genetic Algorithm-------')
import tsp.genetic as genetic

pop_size = 500
crossover_rate = 0.85
mutation_rate=0.15
elitism_count=2
tournament_size=5
# t_ga=[500, 1000] # best performance
t_ga=[100] # limited runtime

res, avg_ga_best, duration_ga = genetic.test_genetic(
    tsplib_test_list, 
    pop_size=pop_size, 
    crossover_rate=crossover_rate, 
    mutation_rate=mutation_rate, 
    elitism_count=elitism_count, 
    tournament_size=tournament_size, 
    t_ga=t_ga, 
    device=device
)

print('Total duration: ', duration_ga)
for i, t in enumerate(t_ga):
    print(f"T={t}, average cost is {avg_ga_best[i]}.")
    res_avg_cost[f'Genetic(T={t})'] = float(avg_ga_best[i])

import csv
RES_DIR = 'results/'
f_out = 'generalization_cost_tsplib100.csv'
test_set = 'TSPLIB100'

with open(RES_DIR + f_out, 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['method', 'test_set', 'average_cost'])
    # Write each key-value pair as a row
    for method, avg_cost in res_avg_cost.items():
        writer.writerow([method, test_set, avg_cost])

-------ACO Model (LS)-------


100%|██████████| 4/4 [00:00<00:00, 14.55it/s]
/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/2635745122.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Total duration:  0.27589893341064453
T=2, average cost is 8.494874954223633.
T=10, average cost is 8.378503799438477.
-------DeepACO (LS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 4
device: cpu


100%|██████████| 4/4 [00:00<00:00, 21.41it/s]


total duration:  0.18797922134399414
T=2, average cost is 8.32490062713623.
T=10, average cost is 8.238510131835938.
-------DeepACO (NLS)-------
checkpoint: ../pretrained/tsp_nls/tsp100-best.pt
number of instances: 4
device: cpu


/var/folders/c_/9pzrss116732p7dxch3kn_bc0000gn/T/ipykernel_51980/2635745122.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net_nls.load_state_dict(torch.load(model_fil

total duration:  1.2342329025268555
T=2, average cost is 8.285407066345215.
T=10, average cost is 8.221202850341797.
GreedyTSP is using: cpu
--------Greedy Algorithm:--------
Total duration:  0.00439000129699707
Average cost is 10.27803460357245.
-------Genetic Algorithm-------


Running Genetic Algorithm...: 100%|██████████| 4/4 [00:11<00:00,  2.82s/it]

Total duration:  11.280088663101196
T=100, average cost is 263.1759033203125.
